# Chatbot de recommandation de films

## Import des bibliothèques

In [ ]:
import numpy as np
import pandas as pd

from google.colab import drive
from google.colab import output

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import vstack

## Création du Chatbot


### Choix du modèle

In [ ]:
#@markdown Choisissez un modèle de la liste suivante:
repo_id = "gorkemgoknar/moviechatbot-mistral-v1" # @param ["TheBloke/Mistral-7B-Instruct-v0.1-GGUF","gorkemgoknar/moviechatbot-mistral-v1","mistralai/Mistral-7B-Instruct-v0.1"]

chatbot_repos = {
    "TheBloke/Mistral-7B-Instruct-v0.1-GGUF": "mistral-7b-instruct-v0.1.Q2_K.gguf",
    "gorkemgoknar/moviechatbot-mistral-v1": "mistral-moviechatbot-q5_K_M.gguf"
}

#### Installation

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 31.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.20-cp310-cp310-manylinux_2_35_x86_64.whl size=7138286 sha256=e67264bafc185209f5f866380cc24d612b1e5d0b9fd64bd14c93ed9e95049934
  Stored in directory: /root/.cache/pip/wheels/ef/f2/d2/0becb03047a348d7bd9a5b91ec88f4654d6fa7d67ea4e84d43
Successfully built llama-cpp-python


In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git
!pip install -r llama.cpp/requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 12959, done.
remote: Counting objects: 100% (3523/3523), done.
remote: Compressing objects: 100% (287/287), done.
remote: Total 12959 (delta 3367), reused 3291 (delta 3235), pack-reused 9436
Receiving objects: 100% (12959/12959), 15.46 MiB | 19.54 MiB/s, done.
Resolving deltas: 100% (9009/9009), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which i

In [ ]:
!pip install huggingface_hub langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00


#### Import du modèle

In [ ]:
if repo_id == "mistralai/Mistral-7B-Instruct-v0.1":
  from huggingface_hub import snapshot_download
  model_id="mistralai/Mistral-7B-Instruct-v0.1"
  snapshot_download(repo_id=model_id, local_dir="mistral-hf",
                  local_dir_use_symlinks=False, revision="main")

In [ ]:
if repo_id != "mistralai/Mistral-7B-Instruct-v0.1":
  from huggingface_hub import hf_hub_download
  model_path = hf_hub_download(repo_id=repo_id, filename=chatbot_repos[repo_id])
else:
  model_path = "artefact/model_llm.gguf"

mistral-moviechatbot-q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

#### Compression du modèle

In [ ]:
compression = "f16" # @param ["f16", "q8_0", "f32"]


In [ ]:
if repo_id == "mistralai/Mistral-7B-Instruct-v0.1":
  !python llama.cpp/convert.py mistral-hf \
    --outfile artefact/model_llm.gguf \
    --outtype $compression

### Génération du modèle

In [ ]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(
  model_path=model_path,
  temperature=0.,
  max_tokens=200,
  top_p=1,
  n_gpu_layers = 100,    # number of layers loaded on the GPU
  verbose=True,          # necessary to load models, else provocate errors
  context_window=2000,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! context_window is not default parameter.
                context_window was transferred to model_kwargs.
                Please confirm that context_window is what you intended.
  warnings.warn(
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


### Prompts

In [ ]:
# This function uses a language model to determine if a given sentence is about a movie.
# It returns True if the sentence is about a movie, and False otherwise.
def sentence_about_movie(llm, sentence, verbose=0):
  question_pattern = """
### Instruction ###
Answer 'Yes' if the following sentence is about movie, else 'No'.

"""
  format = "\n### Response ###\n"
  response = llm(question_pattern+"\n'"+sentence+"'\n"+format)
  if verbose!=0:
    print("About movie: ", response)
  if response[0:2] == "No":
    return False
  return True

# This function uses a language model to extract the name of the movie mentioned in a given sentence.
# If no movie name is mentioned, it returns 'NONE'
def extract_movie_name(llm, sentence, verbose=0):
  question_pattern = """
### Instruction ###
Extract the name of the movie mentionned in the following sentence, else respond 'NONE'.

"""
  format = "\n### Response ###\n"
  response = llm(question_pattern+"\n'"+sentence+"'\n"+format)
  if verbose!=0:
    print("Movie's name: ", response)
  return response

## Import des données

In [ ]:
# drive connection
drive.mount('/content/drive', force_remount=True)

# path when the files are in a shared drive
path = '"/content/drive/Shareddrives/ING3 Projet NLP CY Tech 2023-2024/"'

# move in directories
%cd $path

Mounted at /content/drive
/content/drive/Shareddrives/ING3 Projet NLP CY Tech 2023-2024


In [ ]:
def get_mpst_dataset():
  df = pd.read_csv("dataset/mpst_full_data.csv", sep=",")
  return df

def get_wiki_dataset():
  df_wiki = pd.read_csv("dataset/wiki_movie_plots_deduped.csv")
  return df_wiki

In [ ]:
# @title Choix de la base de données

# @markdown Choisissez la base de données à utiliser parmi celles listées ci-dessous :
# @markdown - [`Wikipedia Movie Plots`](https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots) , base de données de Wikipedia
# @markdown - `Cross dataset` , base de données générée par le croisement de  [Wikipedia Movie Plots](https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots) et [MPST: Movie Plot Synopses with Tags](https://www.kaggle.com/datasets/cryptexcode/mpst-movie-plot-synopses-with-tags)

database_name = "Wikipedia Movie Plots" # @param ["Wikipedia Movie Plots", "Crossed dataset"]


### Prétraitement des données

In [ ]:
# This function vectorizes the text in the dataframe using TF-IDF vectorization.
# It also creates a new feature by repeating the tags based on the length of 'plot_synopsis' and concatenating with the 'plot_synopsis'.
def vectorize_text(df, verbose=0):
  vectorizer = TfidfVectorizer(stop_words="english")
  # Fit the vectorizer on the entire corpus
  vectorizer.fit(df['title'].tolist() + df['plot_synopsis'].tolist())

  # Define a base multiplier : higher value = lower tag importance
  base_multiplier = 8

  # Repeat the tags based on the length of 'plot_synopsis' and concatenate with the 'plot_synopsis'
  synopsis_with_tags = df.apply(lambda row: row['plot_synopsis'] + ' ' + (row['tags'] * (len(row['plot_synopsis'].split()) // base_multiplier)), axis=1)

  df['title_vectorized'] = df['title'].apply(lambda x: vectorizer.transform([x]))
  df['plot_synopsis_vectorized'] = df['plot_synopsis'].apply(lambda x: vectorizer.transform([x]))
  df['plot_synopsis_with_tags_vectorized'] = synopsis_with_tags.apply(lambda x: vectorizer.transform([x]))

  return df, vectorizer

# This function shortens the plot of a movie to a specified number of characters or sentences.
# It uses a language model to generate a summary of the plot.
def summarize_shorten_plot(llm, movie_plot, nb_input_char=200, min_output_char=20, nb_output_sentences=2):
  s_prompt = "[INST] Summarize the movie plot below with "+str(nb_output_sentences)+" sentences."
  e_prompt = "[\INST]"
  if nb_input_char < 10:
    return llm(s_prompt+movie_plot+e_prompt)
  else:
    i = -1
    list_sentences = movie_plot.split("\n")
    while (-i < len(movie_plot)) and (len("\n".join(list_sentences[:i])) > nb_input_char):
      i -= 1
    if (-i < len(movie_plot)) and (i >= min_output_char):
      new_plot = "\n".join(list_sentences[:i])
    else:
      i = -1
      list_sentences = movie_plot.split(".")
      while (-i < len(movie_plot)) and (len(".".join(list_sentences[:i])) > nb_input_char):
        i -= 1
      if (-i < len(movie_plot))and (i >= min_output_char):
        new_plot = ".".join(list_sentences[:i])
      else:
        new_plot = movie_plot[:nb_input_char-1]
        new_plot += "."
    return llm(s_prompt+new_plot+e_prompt)

# This function prepares the Wikipedia dataset for use in the recommendation system.
# It extracts the relevant columns and renames them to match the expected format.
def treat_wiki(df_wiki):
  df = pd.DataFrame()
  df["title"] = df_wiki["Title"]
  df["plot_synopsis"] = df_wiki["Plot"]
  df["tags"] = df_wiki["Genre"]
  return df

# This function merges and prepares the MPST and Wikipedia datasets for use in the recommendation system.
# It matches movies based on title, and uses a language model to generate a summary of the plot if necessary.
def treat_dataset(df_mpst, df_wiki, llm_summarizer=None, verbose=0):
  pretreated_df = pd.DataFrame(columns=["title", "plot_synopsis", "tags"])
  i = 0
  size_df = df_mpst.shape[0]
  for row in df_mpst.values:
    search = df_wiki[row[1] == df_wiki["Title"]]
    if search["Title"].count() == 1:
      pretreated_df.loc[i] = {
          "title": str(row[1]),
          "plot_synopsis": str(search["Plot"]),
          "tags": str(row[3])
      }
      i += 1
    elif (llm_summarizer != None) and (search["Title"].count() < 1):
      # WARNING: This functionnality wasn't tested and might take a while to compute
      if verbose > 0:
        print("STEP ",i+1,"/",size_df,": Summarizing the movie '",row[1],"'.")
      pretreated_df.loc[i] = {
          "title": str(row[1]),
          "plot_synopsis": summarize_shorten_plot(llm_summarizer, search["Plot"], nb_input_char=300),
          "tags": str(row[3])
      }
      i += 1
  return pretreated_df

In [ ]:
if database_name == "Crossed dataset":
  # 1) Generating a new dataset
  df = treat_dataset(get_mpst_dataset(), get_wiki_dataset())
  # 2) Vectorizing the text
  df, vectorizer = vectorize_text(df)
else:
  # 1) Generating a new dataset
  df = treat_wiki(get_wiki_dataset())
  # 2) Vectorizing the text
  df, vectorizer = vectorize_text(df)

KeyboardInterrupt: ignored

In [ ]:
df

## Algorithme de recommandation

In [ ]:
# Function to recommend a movie based on the dataset and the conversation
def content_based_recommendation(input, df, type, vectorizer, top_recommendation_history=[], verbose=0):
    extracted_title, user_sentence = input
    # If the type is 'title', we are looking for a specific movie title
    if type == 'title':
        if extracted_title in df['title'].values:
            if verbose: print("title in DB => by title")
            # Get the vectorized plot synopsis with tags for the extracted title
            input_plot_synopsis_with_tags_vectorized = df[df['title'] == extracted_title]['plot_synopsis_with_tags_vectorized'].values[0]
        else:
            extracted_title_vectorized = vectorizer.transform([extracted_title])
            similarity_scores = cosine_similarity(vstack(df['title_vectorized'].tolist()), extracted_title_vectorized)
            max_similarity_index = np.argmax(similarity_scores)
            max_similarity_score = similarity_scores[max_similarity_index]
            if max_similarity_score > 0.9:
                extracted_title = df.iloc[max_similarity_index]['title']
                if verbose: print("very close title in DB "+extracted_title+"=> by title")
                # Get the vectorized plot synopsis with tags for the matched title
                input_plot_synopsis_with_tags_vectorized = df[df['title'] == extracted_title]['plot_synopsis_with_tags_vectorized'].values[0]
            else:
                if verbose: print("title not in DB => by synopsis")
                # If no match is found, vectorize the user sentence instead
                input_plot_synopsis_with_tags_vectorized = vectorizer.transform([user_sentence])
    # If the type is 'synopsis', we are looking for a movie based on a plot synopsis
    elif type == 'synopsis':
        input_plot_synopsis_with_tags_vectorized = vectorizer.transform([user_sentence])

    similarity_scores = cosine_similarity(vstack(df['plot_synopsis_with_tags_vectorized'].tolist()), input_plot_synopsis_with_tags_vectorized)
    sorted_similarity_scores_and_titles = sorted([(score, title) for score, title in zip(similarity_scores, df['title'])], key=lambda x: x[0], reverse=True)
    # Get the title of the most similar movie that has not been recommended before
    recommended_title = next((title for score, title in sorted_similarity_scores_and_titles if title not in top_recommendation_history), None)
    # Add the recommended title to the history
    top_recommendation_history.append(recommended_title)

    if recommended_title is None:
        return "NONE"
    else:
        # Otherwise, get the plot synopsis of the recommended movie
        selected_synopsis = df[df['title'] == recommended_title]['plot_synopsis'].values[0]
        return recommended_title, selected_synopsis, top_recommendation_history

## Chatbot and Interface setup


In [ ]:
# Function to print on interface
def print_on_interface(interface_file, string):
    if interface_file != "":
        open(interface_file,"a").write(string)
    else:
        print(string)

# Function to show interface
def show_interface(interface_file):
    if interface_file != "":
        output.clear()
        print(open(interface_file,'r').read())

# Function to clear interface
def clear_interface(interface_file):
    if interface_file != "":
        try:
            file = open(interface_file,"x")
        except:
            file = open(interface_file,"r+")
        file.seek(0)
        file.truncate()
        file.close()

# Function to shorten plot
def shorten_plot(movie_plot, nb_input_char=200, min_output_char=20):
  if nb_input_char > len(movie_plot):
    return movie_plot
  else:
    i = -1
    list_sentences = movie_plot.split("\n")
    while (-i < len(movie_plot)) and (len("\n".join(list_sentences[:i])) > nb_input_char):
      i -= 1
    if (-i < len(movie_plot)) and (i >= min_output_char):
      new_plot = "\n".join(list_sentences[:i])
    else:
      i = -1
      list_sentences = movie_plot.split(".")
      while (-i < len(movie_plot)) and (len(".".join(list_sentences[:i])) > nb_input_char):
        i -= 1
      if (-i < len(movie_plot))and (i >= min_output_char):
        new_plot = ".".join(list_sentences[:i])
      else:
        new_plot = movie_plot[:nb_input_char-1]
        new_plot += "."
    return new_plot

# Function to recommend movie
def recommend_movie(llm, sentence, instruction, dt, discussion="", verbose=0, top_recommendation_history = []):
    extracted_title = extract_movie_name(llm, sentence, verbose=verbose)
    type = 'title' if extracted_title != "NONE" else 'synopsis'
    output = content_based_recommendation([extracted_title, sentence], dt, type, vectorizer, top_recommendation_history, verbose = verbose)
    if output != "NONE":
      recommended_title, selected_synopsis, top_recommendation_history = output
      instruction = "[INST] The following is a conversation with an AI that recommends movies. The agent should politely recommend the movie "+recommended_title+" and summerize the following synopsis in at most 4 sentences : "+selected_synopsis+". [\INST]\n"
    line = "Customer: "+sentence+"\nAgent: "
    try:
      return llm(instruction+discussion+line), top_recommendation_history
    except ValueError:
      instruction = "[INST] The following is a conversation with an AI that recommends movies. The agent should politely recommend the movie "+recommended_title+" and summerize the following synopsis in at most 4 sentences : "
      instruction += shorten_plot(selected_synopsis,nb_input_char=1000)+". [\INST]\n"
      return llm(instruction+discussion+line), top_recommendation_history

# Function to pretreat answer
def posttreat_answer(answer, verbose=0):
  if verbose!=0:
    print(answer)
  answer = answer.split("\n")
  i=0
  while (i<len(answer)) and (answer[i]==""):
    i += 1
  if (i<len(answer)):
    return answer[i]
  return "ERROR"

# Function for recommendation chatbot
def recommendation_chatbot(llm, verbose=0):
  if verbose < 1:
    interface_file = "artefact/conversation.txt"
  else:
    interface_file = ""
  clear_interface(interface_file)
  instruction = """
[INST] The following is a conversation with an AI that recommends movies. The agent should politely recommend a movie with its "name" and (genre). [\INST]
"""
  print_on_interface(interface_file,"""-  DISCUSSION WITH CHATBOT -

(Write 'STOP' to end the discussion with the chatbot.)

""")
  print_on_interface(interface_file,"YOU: ")
  show_interface(interface_file)
  user_input = str(input())
  print_on_interface(interface_file,user_input+"\n")
  show_interface(interface_file)
  top_recommendation_history = []
  while user_input != "STOP":
    if sentence_about_movie(llm, user_input, verbose):
      llm_output, top_recommendation_history = recommend_movie(llm, user_input, instruction, df, verbose=verbose, top_recommendation_history = top_recommendation_history)
    else:
      line = "\nCustomer: "+user_input+"\nAgent: "
      reminder_instruction = """
[INST] The following is a conversation with an AI that recommends movies. Customer entered irrelevant input. Remind to the customer your function.  [\INST]
"""
      llm_output = llm(reminder_instruction+line)
    bot_answer = posttreat_answer(llm_output, verbose)
    print_on_interface(interface_file,"BOT: "+bot_answer+"\n")
    print_on_interface(interface_file,"\nYOU: ")
    show_interface(interface_file)
    user_input = str(input())
    print_on_interface(interface_file,user_input+"\n")
    show_interface(interface_file)
  print_on_interface(interface_file,"\n-  END OF DISCUSSION -")
  show_interface(interface_file)

# Function for recommendation chatbot with memory
def recommendation_chatbot_memory(llm, verbose=0):
  if verbose < 1:
    interface_file = "artefact/conversation.txt"
  else:
    interface_file = ""
  clear_interface(interface_file)
  instruction = """
[INST] The following is a conversation with an AI that recommends movies. The agent should politely recommend a movie with its "name" and (genre). [\INST]
"""
  print_on_interface(interface_file,"""-  DISCUSSION WITH CHATBOT -

(Write 'STOP' to end the discussion with the chatbot.)

""")
  discussion = ""
  print_on_interface(interface_file,"YOU: ")
  show_interface(interface_file)
  user_input = str(input())
  print_on_interface(interface_file,user_input+"\n")
  show_interface(interface_file)
  top_recommendation_history = []
  while user_input != "STOP":
    line = "\nCustomer: "+user_input+"\nAgent: "
    if sentence_about_movie(llm, user_input, verbose):
      llm_output, top_recommendation_history = recommend_movie(llm, user_input, instruction, df, verbose=verbose, top_recommendation_history = top_recommendation_history, discussion = discussion)
    else:
      reminder_instruction = """
[INST] The following is a conversation with an AI that recommends movies. Customer entered irrelevant input. Remind to the customer your function.  [\INST]
"""
      llm_output = llm(reminder_instruction+discussion+line)
    bot_answer = posttreat_answer(llm_output, verbose)
    print_on_interface(interface_file,"BOT: "+bot_answer+"\n")
    discussion = line+bot_answer
    print_on_interface(interface_file,"\nYOU: ")
    show_interface(interface_file)
    user_input = str(input())
    print_on_interface(interface_file,user_input+"\n")
    show_interface(interface_file)
  print_on_interface(interface_file,"\n-  END OF DISCUSSION -")
  show_interface(interface_file)

## Chatbot

In [ ]:
# The function does not keep any memory of past messages.
# Each user input is processed independently, and the response is generated based solely on that input.
# This means that the chatbot does not have the ability to maintain context or remember past interactions.
recommendation_chatbot(llm, verbose=1)

In [ ]:
# This function is more sophisticated than recommendation_chatbot because it can maintain context over multiple turns of conversation.
# This can lead to more relevant and coherent responses when the user's inputs are related to each other.
recommendation_chatbot_memory(llm, verbose=0)